# Reto 1: Starbucks
Starbucks comenzó como una empresa minorista de tostado de grano entero, café molido, té y especias con una sola tienda en el mercado Pike Place de Seattle en 1971. La compañía ahora opera más de 24,000 tiendas minoristas en 70 países.

Para este reto usarás un conjunto de datos que incluye registros sobre cada tienda Starbucks o subsidiaria actualmente en funcionamiento.

1. Analiza los datos, limpia los datos en caso de ser necesario.

2. Elige MySQL o MongoDB y crea una base de datos para el conjunto de datos del reto.

3. Carga los datos en la base de datos que elegiste y revisa que éstos se muestren correctamente.

4. Usando la latitud y longitud de tu posición actual, encuenta el Starbucks más cercano a tu posición. Para conocer tu posición actual puedes usar Google Maps para, sólo debes copiar los datos de la URL.

In [2]:
import pymongo
import re
import pandas as pd
client = pymongo.MongoClient(f"mongodb://{user}:{password}@{proyecto}-shard-00-00.mmsfi.mongodb.net:27017,{proyecto}-shard-00-01.mmsfi.mongodb.net:27017,{proyecto}-shard-00-02.mmsfi.mongodb.net:27017/test?ssl=true&replicaSet=atlas-11i31k-shard-0&authSource=admin&retryWrites=true&w=majority")
db = client.test

Aqui podemos ingresar diferentes coordenadas para encontrar el establecimiento mas cercano, por ejemplo pongamos las coordenadas de BEDU.

In [3]:
lat = 19.4218099
long = -99.1613632

In [4]:
result = client['starbucks']['starbucks'].aggregate([
    {
        '$addFields': {
            'actualLat': lat, 
            'actualLong': long
        }
    }, {
        '$addFields': {
            'distanceToStore': {
                '$sqrt': {
                    '$add': [
                        {
                            '$pow': [
                                {
                                    '$subtract': [
                                        '$actualLat', '$Latitude'
                                    ]
                                }, 2
                            ]
                        }, {
                            '$pow': [
                                {
                                    '$subtract': [
                                        '$actualLong', '$Longitude'
                                    ]
                                }, 2
                            ]
                        }
                    ]
                }
            }
        }
    }, {
        '$match': {
            'distanceToStore': {
                '$gte': 0
            }
        }
    }, {
        '$sort': {
            'distanceToStore': 1
        }
    }, {
        '$limit': 1
    }, {
        '$project': {
            '_id': 0, 
            'Store Name': 1, 
            'Street Address': 1, 
            'City': 1, 
            'Country': 1, 
            'distanceToStore': 1
        }
    }
])

In [5]:
x = list(result)
df = pd.DataFrame(x)
df

,Store Name,Street Address,City,Country,distanceToStore
0,Plaza Luis Cabrera,"Plaza Luis Cabrera # 5 Local 1, Col Roma",Mexico City,MX,0.002266
